In [13]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [14]:
# with open("2022-06-26-SC-Earthquake.json", "r", encoding="utf-8") as f:
with open("./data/2022-09-03-SC-Earthquake.geojson", "r", encoding="utf-8") as f:
    SC_eq_list = json.load(f)

In [15]:
# SC_eq_list is a python list of dictionaries
# flattens sc_eq_List into sc_eq_data, a pandas dataframe
sc_eq_data = pd.json_normalize(SC_eq_list['features'])

In [16]:
eq_df = sc_eq_data[['properties.mag', 'properties.felt', 'properties.place', 'properties.time', 
                    'properties.title', 'geometry.coordinates']]

# since the coordinates list was flattened above, convert it back to a list with each element as a column
# in the coord_df dataframe along with column names
coord_df = pd.DataFrame(sc_eq_data['geometry.coordinates'].tolist(), columns=['lon', 'lat', 'depth(km)'])

# concatenate coord_df to sc_eq_data
eq_df = pd.concat([eq_df, coord_df], axis=1)

eq_df['Event_Date'] = pd.to_datetime(eq_df['properties.time'], unit="ms")\
                             .dt.tz_localize('UTC')\
                             .dt.tz_convert('America/New_York').dt.date

eq_df['Event_Time'] = pd.to_datetime(eq_df['properties.time'], unit="ms")\
                             .dt.tz_localize('UTC')\
                             .dt.tz_convert('America/New_York').dt.time

In [17]:
# remove decimal portion of the seconds part of the time

import datetime
for x in range(len(eq_df['Event_Time'])):
    eq_df.loc[x, 'Event_Time'] = eq_df['Event_Time'][x].replace(microsecond = 0)

In [18]:
eq_df = eq_df.copy()

In [19]:
eq_df = eq_df.rename(columns={'properties.mag': 'Magnitude',
                              'properties.felt': 'Felt',
                              'properties.place': 'Location',
                              'properties.time': 'Event_DT', 
                              'properties.title': 'Event_Title', 
                              'geometry.coordinates': 'Coordinates',
                              'lon': 'Longitude', 'lat': 'Latitude', 'depth(km)': 'Depth(km)',
                              'Event_Date': 'Event_Date', 'Event_Time': 'Event_Time'})

In [20]:
# The eq_df['Felt'] series had/may have null values; The following expression finds those rows
# with a null value in the 'Felt' column and assigns it to 0 and casts it to an integer dtype.

eq_df['Felt'] = eq_df['Felt'].fillna(0).astype('int')

In [21]:
eq_df.head().style.set_caption("South Carolina Earthquakes - Dec. - 2021 to Now").hide(axis="index")\
                  .format({"Magnitude": "{:.2f}",
                           "Felt": "{:d}",
                           "Location": "{:s}",
                           "Event_DT": "{:d}",
                           "Event_Title": "{:s}",
                           "Latitude": "{:.6f}",
                           "Longitude": "{:.6f}", 
                           "Depth(km)": "{:.2f}", 
                           "Event_Date": "{:%Y-%m-%d}",
                           "Event_Time": "{:%H:%M:%S}"})

Magnitude,Felt,Location,Event_DT,Event_Title,Coordinates,Longitude,Latitude,Depth(km),Event_Date,Event_Time
1.45,0,"3 km S of Centerville, South Carolina",1662081817890,"M 1.5 - 3 km S of Centerville, South Carolina","[-80.1761667, 32.9653333, 0.74]",-80.176167,32.965333,0.74,2022-09-01,21:23:37
1.62,17,"7 km SSW of Lugoff, South Carolina",1661625694850,"M 1.6 - 7 km SSW of Lugoff, South Carolina","[-80.7105, 34.1655, 3.09]",-80.710500,34.165500,3.09,2022-08-27,14:41:34
2.11,11,"6 km SE of Homeland Park, South Carolina",1660802433420,"M 2.1 - 6 km SE of Homeland Park, South Carolina","[-82.6106667, 34.4325, 0.03]",-82.610667,34.432500,0.03,2022-08-18,02:00:33
1.53,11,"7 km E of Elgin, South Carolina",1660555959540,"M 1.5 - 7 km E of Elgin, South Carolina","[-80.7128333, 34.1606667, 3.59]",-80.712833,34.160667,3.59,2022-08-15,05:32:39
1.64,10,"9 km W of Boykin, South Carolina",1660024791740,"M 1.6 - 9 km W of Boykin, South Carolina","[-80.6846667, 34.1086667, 6.85]",-80.684667,34.108667,6.85,2022-08-09,01:59:51


### Plot Coordinates via Plotly

In [22]:
# read in mapbox access token
mapbox_access_token = open(".mapbox_token").read()

m = list(eq_df['Event_Date'])
n = list(eq_df['Event_Time'])
k = list(eq_df['Magnitude'])
l = list(eq_df['Depth(km)'])
t = list(eq_df['Event_Title'])

In [23]:
# create a 4 row numpy array that contains - row 0: dates, row 1: times, row 2: magnitude values, row 3: depth
# the customdata parameter is assigned the numpy array so we can have those values displayed in the hover
nk = np.empty(shape=(len(k),5,1), dtype='object')
nk[:,0] = np.array(m).reshape(-1,1)  # row with dates
nk[:,1] = np.array(n).reshape(-1,1)  # row with times
nk[:,2] = np.array(k).reshape(-1,1)  # row with magnitude values
nk[:,3] = np.array(l).reshape(-1,1)  # row with depth values
nk[:,4] = np.array(t).reshape(-1,1)  # row with event titles

In [24]:
fig = go.Figure(go.Scattermapbox(name="",
                                 lat=list(eq_df["Latitude"]),
                                 lon=list(eq_df["Longitude"]),
                                 mode='markers',
#                                 marker=go.scattermapbox.Marker(size=12, color="#7FFFD4"),
                                 marker=go.scattermapbox.Marker(size=12,
                                                                color=eq_df['Magnitude'],
                                                                colorscale='jet'),
                                 text=list(eq_df["Location"]),
                                 customdata=nk,
                                 hovertemplate = "Event Title: %{customdata[4]}<br>" +
                                                 "Event Date: %{customdata[0]}<br>" + 
                                                 "Event Time: %{customdata[1]}<br>" +
                                                 "Location: %{text}<br>" +
                                                 "Magnitude: %{customdata[2]}<br>" + 
                                                 "lat:  %{lat},  " + "lon:  %{lon}<br>" +
                                                 "Depth(km):  %{customdata[3]}"))

fig.update_layout(mapbox_style="dark",
                  title_font_color = "#00FF7F",
                  title_text="South Carolina Earthquakes - Dec. - 2021 to Present",
                  paper_bgcolor = "#8B4513",
                  autosize=True,
                  height=510,
                  margin=dict(t=50, b=5, l=5, r=5),
                  hovermode='closest',
                  mapbox=dict(accesstoken=mapbox_access_token,
                              bearing=0,
                              center=dict(lat=34.170983, lon=-80.794252),
                              pitch=0,
                              zoom=11),
                  hoverlabel=dict(bgcolor="white",
                                  font_size=15,
                                  font_family="Rockwell"),
                  showlegend = False)

fig.show()